In [1]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

2025-04-07 15:55:19,501 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Carregar o arquivo CSV
df_times = pd.read_excel("times.xlsx")

# Ver os dados carregados
display(df_times.head())

,Nome
0,FBC Colorado
1,Texas Club 2025
2,Real SCI
3,Gig@ntte
4,PraiaBravaFC


### Buscar IDs dos times no Cartola

In [3]:
nomes_times = ['FBC Colorado', 'Texas Club 2025', 'Real SCI', 'Gig@ntte', 'PraiaBravaFC', 'OlhaEleAiF.C!', 'Gremiomaniasm', 'Sport Clube PAIM', 'PUXE FC', 'RS Expressões da Arte',
               'ZIVI FC', 'O clube do povo Itaqui/Rss', 'F.C. Rei Das Copas', 'Rolo Compressor ZN', 'Taura da Fronteira FCIII', 'Eleis-Itaqui', 'KING LEONN', 'Laranjja Mecannica', 
               'Fedato Futebol Clube', 'Perronee F.C', 'Pity10', 'pra sempre imortal fc', 'RHANKA DENTY FC25', 'TEAM LOPES 99', 'pura bucha /botafogo', 'cartola scheuer', 'Remo Santo Ângelo', 
               'Analove10 ITAQUI GRANDE!!', 'DM Studio', 'lsauer fc', 'VASCO MARTINS FC', 'KP JUV.', 'BORGES ITAQUI F.C.', 'Profit Soccer', 'Tabajara de Inhaua FC2', 'TIGRE LEON', 
               'S.E.R. GRILLO', 'seralex', 'E.C. Bororé', 'MAFRA MARTINS FC', 'BordonFC', 'Tatols Beants F.C', 'FIGUEIRA DA ILHA', 'MauHumor F.C.', 'A Lenda Super Vascão f.c', 
               'TATITTA FC', 'HS SPORTS F.C', 'Dom Camillo68', 'mercearia Estrela', 'CosmoCity ZO', 'clarinvalau fc', 'Grêmio imortal 37', 'SERGRILLO', 'Super Vasco f.c', 
               'A Lenda Super Vasco F.c', 'Paulo Virgili FC', 'CALOMBO ITAQUI RS', 'Luis lemes inter', 'emer jr fc', 'Cril Futebol Club', 'ITAQUI F.C.', 'TORRESMO COM PINGA', 
               'Lá do Itaqui', 'FC Los Castilho', 'KillerColorado', "Atlético Colorado 2021", "Time do S.A.P.O", "SISO FC25", "Grêmio imortal 37", "BORGES ITAQUI F.C."]

# Dicionário para armazenar os IDs dos times
ids_times = {}

# Função robusta para buscar ID do time por nome
def buscar_id_time(nome_time):
    try:
        times = api.times(query=nome_time)
        nomes_api = [time.nome for time in times]

        # Comparação flexível (removendo acentos, pontos, etc)
        nome_base = nome_time.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip()
        nomes_api_base = [n.lower().replace(".", "").replace("fc", "").replace("f.c", "").strip() for n in nomes_api]

        nome_proximo = get_close_matches(nome_base, nomes_api_base, n=1, cutoff=0.6)

        if nome_proximo:
            idx = nomes_api_base.index(nome_proximo[0])
            return times[idx].id
    except cartolafc.errors.CartolaFCError as e:
        print(f"Erro ao buscar ID para o time {nome_time}: {e}")
    return None

# Buscar IDs automaticamente, sem sobrescrever os que já estiverem no dicionário
nao_encontrados = []

for nome in nomes_times:
    if nome not in ids_times:  # Protege os manuais
        time_id = buscar_id_time(nome)
        if time_id:
            ids_times[nome] = time_id
        else:
            nao_encontrados.append(nome)

# ➕ Correções manuais
# ids_times["BORGES ITAQUI F.C."] = 3914981
# ids_times["Time do S.A.P.O"] = 12345678  # Substitua pelo ID real

# Mostrar resultado
print("IDs encontrados:", len(ids_times))
print("Times não encontrados automaticamente:", nao_encontrados)

IDs encontrados: 68
Times não encontrados automaticamente: []


### Gerar DataFrame com nome do time, ID e URL

In [4]:
# Gerar DataFrame com nome do time, ID e URL
df_urls = pd.DataFrame([
    {"Nome do Time": nome, "ID do Time": time_id, "Link do Time": f"https://cartola.globo.com/#!/time/{time_id}"}
    for nome, time_id in ids_times.items()
])

# Caminho e nome do arquivo Excel
caminho_excel = "links_times_cartola_liga_serie_A.xlsx"

# Salvar o DataFrame em Excel
df_urls.to_excel(caminho_excel, index=False)

print(f"✅ Arquivo salvo com sucesso: {caminho_excel}")


# Exibir como tabela
display(df_urls.head())

✅ Arquivo salvo com sucesso: links_times_cartola_liga_serie_A.xlsx


,Nome do Time,ID do Time,Link do Time
0,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
1,Texas Club 2025,1273719,https://cartola.globo.com/#!/time/1273719
2,Real SCI,3246608,https://cartola.globo.com/#!/time/3246608
3,Gig@ntte,18421230,https://cartola.globo.com/#!/time/18421230
4,PraiaBravaFC,47546604,https://cartola.globo.com/#!/time/47546604


### Gerar o dicionário ID -> Nome do Time

In [5]:
# Gerar o dicionário ID -> Nome do Time
nomes_por_id = dict(zip(df_urls["ID do Time"], df_urls["Nome do Time"]))

# Mostrar parte do dicionário
dict(list(nomes_por_id.items())[:5])

{186283: 'FBC Colorado',
 1273719: 'Texas Club 2025',
 3246608: 'Real SCI',
 18421230: 'Gig@ntte',
 47546604: 'PraiaBravaFC'}

In [6]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=1).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(1, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes

def gerar_df_pontuacoes(api, ids_times):
    rodada_atual = api.mercado().rodada_atual
    total_rodadas = 38

    if not campeonato_comecou(api, ids_times):
        print("📌 O campeonato ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(1, total_rodadas + 1)])
        df[:] = 0
    else:
        df = pd.DataFrame()
        for nome, time_id in ids_times.items():
            pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
            df[nome] = pd.Series(pontuacoes)
        df = df.transpose()
        df.columns = [f'Rodada {i}' for i in range(1, rodada_atual)]
        df.loc['Lider_Rodada'] = df.idxmax()
    
    return df

In [7]:
ids_times = {v: k for k, v in nomes_por_id.items()}

df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

display(df_pontuacoes.head(20))

,Rodada 1,Rodada 2
FBC Colorado,64.00,72.69
Texas Club 2025,67.69,85.99
Real SCI,104.09,75.69
Gig@ntte,100.34,86.39
PraiaBravaFC,96.39,96.19
OlhaEleAiF.C!,96.19,102.99
Gremiomaniasm,95.23,59.22
Sport Clube PAIM,93.85,67.39
PUXE FC,84.69,95.29
RS Expressões da Arte,83.99,92.82


### Função para definir a classificação dos times

In [8]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):
    """
    Classificação dos grupos com base nos confrontos e nas pontuações do Cartola.

    Retorna:
    - df_resultado: classificação geral
    - df_resultado_por_grupo: dicionário com classificação separada por grupo
    """
    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        grupo = confronto["Grupo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        if pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante):
            continue

        for time in [mandante, visitante]:
            if grupo not in estatisticas:
                estatisticas[grupo] = {}
            if time not in estatisticas[grupo]:
                estatisticas[grupo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0, "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[grupo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[grupo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[grupo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[grupo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[grupo][mandante]["Pontos"] += 3
            estatisticas[grupo][mandante]["Vitórias"] += 1
            estatisticas[grupo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[grupo][visitante]["Pontos"] += 3
            estatisticas[grupo][visitante]["Vitórias"] += 1
            estatisticas[grupo][mandante]["Derrotas"] += 1
        else:
            estatisticas[grupo][mandante]["Pontos"] += 1
            estatisticas[grupo][visitante]["Pontos"] += 1
            estatisticas[grupo][mandante]["Empates"] += 1
            estatisticas[grupo][visitante]["Empates"] += 1

    # Montar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Grupo": grupo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        })
        for grupo, times in estatisticas.items()
    ], ignore_index=True)


    df_resultado = df_resultado.sort_values(
        by=["Grupo", "Pontos", "Total Cartola"],
        ascending=[True, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Grupo")\
    .cumcount() + 1

    df_resultado_por_grupo = {
        grupo: df_resultado[df_resultado["Grupo"] == grupo] for grupo in df_resultado["Grupo"].unique()
    }

    return df_resultado, df_resultado_por_grupo

In [9]:
# 1. Carregar confrontos
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# 2. Renomear colunas se necessário
df_confrontos.rename(columns={
    "Time A": "Mandante_Nome",
    "Time B": "Visitante_Nome"
}, inplace=True)

# 3. Adicionar coluna de grupo fixo
df_confrontos["Grupo"] = "Série A"

display(df_confrontos.head())

# 4. Calcular classificação
df_classificacao, _ = classificacao_por_grupo(df_confrontos, df_pontuacoes)

# 5. Ver resultado
display(df_classificacao.head())

,Rodada,Confronto,Mandante_Nome,Visitante_Nome,ID A,ID B,Grupo
0,1,1,FBC Colorado,Tatols Beants F.C,6,15,Série A
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17,Série A
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10,Série A
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16,Série A
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1,Série A


,Grupo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
5,Série A,OlhaEleAiF.C!,6,2,0,0,199.18,148.97,50.21,1
3,Série A,Time do S.A.P.O,6,2,0,0,176.94,157.08,19.86,2
6,Série A,Atlético Colorado 2021,6,2,0,0,163.53,143.88,19.65,3
19,Série A,SISO FC25,6,2,0,0,161.18,136.04,25.14,4
13,Série A,cartola scheuer,6,2,0,0,159.48,145.27,14.21,5


In [10]:
# 1. Times únicos nos confrontos
times_confrontos = pd.unique(df_confrontos[["Mandante_Nome", "Visitante_Nome"]].values.ravel())

# 2. Times únicos na classificação final
times_classificados = df_classificacao["Nome do Time"].unique()

# 3. Ver quem está nos confrontos mas não foi classificado
faltando = set(times_confrontos) - set(times_classificados)

print("❌ Times que estão nos confrontos, mas faltam na classificação:")
print(faltando)

# df_pontuacoes.loc[["Time do S.A.P.O", "BORGES ITAQUI F.C."]]


❌ Times que estão nos confrontos, mas faltam na classificação:
set()


In [11]:
# Exportar resultado em CSV
df_classificacao.to_csv("classificacao_serie_a.csv", index=False)

# Carregar o arquivo CSV enviado
df_classificacao = pd.read_csv("classificacao_serie_a.csv")

# Renomear colunas para os nomes usados no JavaScript
df_classificacao.rename(columns={
    "Grupo": "grupo",
    "Nome do Time": "nome",
    "Pontos": "pontos",
    "Vitórias": "vitorias",
    "Empates": "empates",
    "Derrotas": "derrotas",
    "Total Cartola": "totalCartola",
    "Cartola Sofrido": "cartolaSofrido",
    "Saldo Cartola": "saldoCartola",
    "Posição": "posicao"
}, inplace=True)

# Agrupar por grupo (apenas Série A nesse caso)
classificacao_por_grupo = {}
for grupo, dados in df_classificacao.groupby("grupo"):
    classificacao_por_grupo[grupo] = dados.drop(columns="grupo").to_dict(orient="records")

# Salvar como arquivo JavaScript
caminho_saida = "classificacao_serie_A.js"
with open(caminho_saida, "w", encoding="utf-8") as f:
    f.write(f"const classificacaoSerieA = {json.dumps(classificacao_por_grupo, indent=2, ensure_ascii=False)};")

caminho_saida

'classificacao_serie_A.js'

In [12]:
import pandas as pd
import json

# Carregar CSV
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

# Limpar espaços nas colunas
df_confrontos.columns = df_confrontos.columns.str.strip()

# Renomear colunas
df_confrontos.rename(columns={
    "Rodada": "rodada",
    "Confronto": "confronto",
    "Time A": "mandante_nome",
    "Time B": "visitante_nome",
    "ID A": "mandante_id",
    "ID B": "visitante_id"
}, inplace=True)

# Adicionar grupo fixo
df_confrontos["grupo"] = "Série A"

# Gerar estrutura para JS
confrontos_formatado = []
for _, row in df_confrontos.iterrows():
    confronto = {
        "rodada": int(row["rodada"]),
        "confronto": int(row["confronto"]),
        "grupo": row["grupo"],
        "mandante": {"id": row["mandante_id"], "nome": row["mandante_nome"]},
        "visitante": {"id": row["visitante_id"], "nome": row["visitante_nome"]}
    }
    confrontos_formatado.append(confronto)

# Salvar como arquivo JS
js_output = f"const confrontosFase1 = {json.dumps(confrontos_formatado, indent=2, ensure_ascii=False)};"
with open("confrontos_serie_A.js", "w", encoding="utf-8") as f:
    f.write(js_output)

print("✅ Arquivo confrontos_serie_A.js gerado com sucesso!")


✅ Arquivo confrontos_serie_A.js gerado com sucesso!


In [13]:
# Carregar o arquivo CSV enviado
df_confrontos = pd.read_csv("confrontos_serie_A.csv")

display(df_confrontos.head())

,Rodada,Confronto,Time A,Time B,ID A,ID B
0,1,1,FBC Colorado,Tatols Beants F.C,6,15
1,1,2,BORGES ITAQUI F.C.,Time do S.A.P.O,3,17
2,1,3,Fedato Futebol Clube,OlhaEleAiF.C!,7,10
3,1,4,Atlético Colorado 2021,Texas Club 2025,2,16
4,1,5,Tabajara de Inhaua FC2,Analove10 ITAQUI GRANDE!!,14,1


In [14]:
def gerar_resultados_serie_a(df_confrontos, df_pontuacoes, grupo="Série A"):
    """
    Gera um DataFrame com os resultados dos confrontos da Série A por rodada.
    """
    resultados = []

    for _, row in df_confrontos.iterrows():
        rodada = row["Rodada"]
        mandante = row["Time A"]
        visitante = row["Time B"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "grupo": grupo,
            "rodada": rodada,
            "mandante": {
                "nome": mandante,
                "pontos": pontos_mandante
            },
            "visitante": {
                "nome": visitante,
                "pontos": pontos_visitante
            }
        })

    return pd.DataFrame(resultados)


In [15]:
df_resultados = gerar_resultados_serie_a(df_confrontos, df_pontuacoes)

# Exportar para .js
import json

with open("resultados_serie_A.js", "w", encoding="utf-8") as f:
    f.write("const resultadosFase1 = ")
    f.write(json.dumps(df_resultados.to_dict(orient="records"), indent=2, ensure_ascii=False))
    f.write(";")

print("✅ Arquivo resultados_serie_A.js gerado com sucesso.")


✅ Arquivo resultados_serie_A.js gerado com sucesso.
